In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import numpy as np
import random
from abc import ABC, abstractmethod
import pandas as pd
import copy

# Preface 
***
* This notebook will go through employing object-oriented programming (OOP) to design neural network classes from the ground up, then instantiating and designing a neural network model for image classification .

* More specifically, it aims to solve a multi-class classification problem, handwritten digit recognition, with the [MNIST dataset](https://www.kaggle.com/competitions/digit-recognizer/data).

* [GitHub repository](http://)


# Contents:
***
* **Data Preparation**
    * [Preparing the MNIST Dataset](#mnist-dataset)
* **Data Augmentation**
    * [Augmenting the MNIST Dataset](#augmenting-the-mnist-dataset)
* **Functions**
    * [Activation Functions](#activation-functions)
    * [Cost Functions](#cost-functions)
    * [Weight Initialization Methods](#weight-Initialization-methods)
    * [Bias Initialization Methods](#bias-initialization-methods)
* **Core Functionality**
    * [Layer Classes](#layer-classes)
    * [Neural Network Classes](#neural-network-classes)
* **MNIST Digit Classifier**
    * [Creating a Digit Classifier](#creating-a-digit-classifier)
* **Miscellaneous**
    * [Comparing `_back_prop` to TensorFlow's backpropagation implementation](#Comparing-back-prop-to-tensorflow)

# <a name="mnist-dataset"></a> Preparing the MNIST Dataset
***
* The [MNIST Dataset](https://www.kaggle.com/competitions/digit-recognizer/data) is one of the most famous datasets in machine learning, serving as a benchmark for image classification.

* It consists of 28 by 28 pixel images of handwritten digits (0-9).


* The data is split into two parts: 
    * The training set, consisting of 42000 training examples, and
    * The test set, consisting of 28000 examples.
    
    
* However, since the test set does not contain labels, we will be using only the training set. We will split the 42000 training examples as follows:
    * **Training set (70%)**: 29400 examples
    * **Validation set (15%)**: 6300 examples
    * **Test set (15%)**: 6300 examples
    
    
* **Pre-processing**:
    * **`x` (input image)** : Extract the pixel values from the `csv` file. Each image is represented by a 784 ($28 \times 28$) element vector.
    * **`y` (label)** : We want the labels to be one-hot-encoded in order to match the output of the neural network.
    * We combine the list of image vectors and the list of labels into a list of tuples. This allows us to shuffle the dataset without mismatching the images and labels.
    
    
* **Objective**: To build a neural network using our custom built classes and functions that can classify these handwritten digits well.

In [3]:
train_data = pd.read_csv('../input/digit-recognizer/train.csv')
test_data = pd.read_csv('../input/digit-recognizer/test.csv')

In [4]:
data_x = train_data.drop(columns=['label']).values / 255.0
data_y = train_data['label'].values
one_hot_data_y = np.array([np.eye(10)[int(y)].reshape(10,1) for y in data_y])
data_set = list(zip(data_x, one_hot_data_y))
data_set = np.array(data_set, dtype=object)

In [5]:
# Specify a seed for reproducible results

def shuffle_set(data_set, seed=None):
    if seed:
        np.random.seed(seed)
    np.random.shuffle(data_set)


shuffle_set(data_set)

In [6]:
# Training, validation, test set split

training_set = data_set[12600:]
test_set = data_set[:6300]
validation_set = data_set[6300:12600]

len(training_set), len(validation_set), len(test_set)

(29400, 6300, 6300)

In [7]:
# Visualizing one of the training images

def visualize_input(x):
    df = pd.DataFrame(x.reshape(28,28))
    return df.style.set_properties(**{'font-size':'2pt'}).background_gradient('Greys')

    
visualize_input(training_set[0][0])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.074510,0.435294,0.752941,0.996078,0.996078,0.996078,0.396078,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027451,0.749020,0.878431,0.992157,0.992157,0.992157,0.960784,0.980392,0.407843,0.000000,0.000000,0.015686,0.345098,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.152941,0.886275,0.992157,0.992157,0.952941,0.592157,0.349020,0.070588,0.525490,0.949020,0.086275,0.027451,0.639216,0.945098,0.164706,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.101961,0.858824,0.992157,0.984314,0.349020,0.113725,0.000000,0.000000,0.007843,0.533333,0.972549,0.254902,0.784314,0.992157,0.658824,0.035294,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176471,0.952941,0.992157,0.607843,0.000000,0.000000,0.000000,0.000000,0.000000,0.352941,0.992157,0.992157,0.992157,0.992157,0.192157,0.000000,0.000000,0.000000,0.000000,0.000000


# <a name="augmenting-the-mnist-dataset"></a> Augmenting the MNIST Dataset
***

In [8]:
'''
training_example: a 2-element numpy array [x, y] where x is the 
input image, and y is its label.

translation, rotate, and scaling return the augmented training_example.
'''

def translation(x_shift, y_shift, training_example):
    input_image_vector = training_example[0]
    image_length = int(np.sqrt(len(input_image_vector)))
    input_image = input_image_vector.reshape(image_length, image_length)
    new_image = np.zeros((image_length, image_length))
    for y_new in range(image_length):
        for x_new in range(image_length):
            x_old = x_new - x_shift
            y_old = y_new - y_shift
            if (0 <= x_old < image_length) and (0 <= y_old < image_length):
                new_image[y_new, x_new] = input_image[y_old, x_old]
    return np.array([new_image.reshape(-1), training_example[1]], dtype=object)

def rotation(degree, training_example):
    radians = degree * (np.pi / 180)
    input_image_vector = training_example[0]
    image_length = int(np.sqrt(len(input_image_vector)))
    input_image = input_image_vector.reshape(image_length, image_length)
    new_image = np.zeros((image_length, image_length))
    center = image_length / 2
    for y_new in range(image_length):
        for x_new in range(image_length):
            x_old = (x_new - center) * np.cos(-radians) - (y_new - center) * np.sin(-radians) + center
            y_old = (x_new - center) * np.sin(-radians) + (y_new - center) * np.cos(-radians) + center
            x_old, y_old = int(round(x_old)), int(round(y_old))
            if (0 <= x_old < image_length) and (0 <= y_old < image_length):
                new_image[y_new, x_new] = input_image[y_old, x_old]
    return np.array([new_image.reshape(-1), training_example[1]], dtype=object)
    
def scaling(factor, training_example):
    input_image_vector = training_example[0]
    image_length = int(np.sqrt(len(input_image_vector)))
    input_image = input_image_vector.reshape(image_length, image_length)
    new_image = np.zeros((image_length, image_length))
    center = image_length / 2
    inverse_factor = 1 / factor
    for y_new in range(image_length):
        for x_new in range(image_length):
            x_old = (x_new - center) * inverse_factor + center
            y_old = (y_new - center) * inverse_factor + center
            x_old, y_old = int(round(x_old)), int(round(y_old))
            if (0 <= x_old < image_length) and (0 <= y_old < image_length):
                new_image[y_new, x_new] = input_image[y_old, x_old]
    return np.array([new_image.reshape(-1), training_example[1]], dtype=object)

def apply_random_augmentation(training_example):
    transformation = random.choice([translation, rotation, scaling])
    if (transformation == translation):
        x_shift = random.randint(-4, 4)
        y_shift = random.randint(-4, 4)
        return transformation(x_shift, y_shift, training_example)
    elif (transformation == rotation):
        degree = random.randint(-20, 20)
        return transformation(degree, training_example)
    else:
        scaling_factor = random.randint(8, 12) / 10
        return transformation(scaling_factor, training_example)
    
def augment_training_images(training_set, augmented_fraction):
    print(f"training_set.shape: {training_set.shape}\n")
    num_to_augment = int(augmented_fraction * len(training_set))
    augmented_set = []
    for training_example in training_set[0:num_to_augment]:
       augmented_set.append(apply_random_augmentation(training_example))
    augmented_training_set = np.concatenate([training_set, np.array(augmented_set)], axis=0)
    print(f"augmented_training_set.shape: {augmented_training_set.shape}\n")
    return augmented_training_set

# <a name="weight-Initialization-methods"></a> Weight Initialization Methods
***
* Specific ways of randomly initializing weight values address the problem of vanishing/exploding gradients, and allows for more effective training.


* The initialization method used depends on the activation function used.
    * For example, He Initialization is used with the ReLU activation funtcion.

In [9]:
def weight_init_function(fn):
    function = WEIGHT_INITS.get(fn.lower())
    if function is None:
        raise ValueError(f"unknown function: {fn}")
    return function

def standard_normal_weights(units, prev_units):
    return np.random.randn(units, prev_units)

def he_initialization(units, prev_units):
    return np.random.randn(units, prev_units) * np.sqrt(2 / prev_units)

WEIGHT_INITS = {
    "standard_normal_weights": standard_normal_weights,
    "he_initialization": he_initialization
}

# <a name="bias-initialization-methods"></a> Bias Initialization Methods
***
* Similarly to weights, specific ways of initializing a model's bias parameters will impact the model's performance.

In [10]:
def bias_init_function(fn, constant=None):
    if fn.lower() == 'small_constant' and constant is not None:
        return lambda units: small_constant(units, constant)
    else:
        function = BIAS_INITS.get(fn.lower())
        if function is None:
            raise ValueError(f"unknown function: {fn}")
        return function

def zero_initialization(units):
    return np.zeros((units, 1))

def small_constant(units, constant=0.01):
    return np.repeat(constant, units).reshape(units, 1)

def standard_normal_biases(units):
    return np.random.randn(units, 1)

BIAS_INITS = {
    "zero_initialization": zero_initialization,
    "small_constant": small_constant,
    "standard_normal_biases": standard_normal_biases
}

# <a name="activation-functions"></a> Activation Functions
***
* Activation functions introduce non-linearity into a neural network, which is what gives them the ability to approximate just about any function.

* Along with the activation functions, we will need their derivatives in order to implement the backpropagation algorithm.

In [11]:
# Returns a function object
def activation_function(fn):
    function = ACTIVATION_FUNCTIONS.get(fn.lower())
    if function is None:
        raise ValueError(f"unknown function: {fn}")
    return function

def activation_derivative(fn, z):
    derivative = ACTIVATION_DERIVATIVES.get(fn)
    if derivative is None:
        raise ValueError(f"unknown derivative for function: {fn}")
    return derivative(z)
    
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def relu(z):
    return np.maximum(0, z)

def linear(z):
    return z

def softmax(z):
    z_stable = z - max(z)
    return np.exp(z_stable) / sum(np.exp(z_stable))

def sigmoid_derivative(z):
    sig = sigmoid(z)
    return sig * (1 - sig)

def relu_derivative(z):
    return np.where(z > 0, 1, 0)
    
def linear_derivative(z):
    return 1.0

ACTIVATION_FUNCTIONS = {
    'relu': relu,
    'sigmoid': sigmoid,
    'linear': linear,
    'softmax': softmax
}

ACTIVATION_DERIVATIVES = {
    relu: relu_derivative,
    sigmoid: sigmoid_derivative,
    linear: linear_derivative
}

# <a name="cost-functions"></a> Cost Functions
***
* Cost functions are crucial in the training process. They are a measure of how well the model is performing on the training data, and therefore tells us how we should adjust our weights and biases.

* Just like with activation functions, we need their derivatives in order to implement the backpropagation algorithm. More specifically, we need it to calculate the error term for the output layer:

    $$\delta^L = \nabla_aC \odot \sigma'(z^L)$$
    
    * $\nabla_aC$ is the gradient of the cost function with respect to the output layer activations.
    * $\sigma'(z^L)$ is the derivative of the activation function of the output layer applied element-wise to the weighted sums of the output layer.
    
    
<br>
 
* However, there is a "special case". For multiclass classification problems where we will be using the cross-entropy loss function with a softmax activation function for the output layer, the above expression for $\delta^L$ simplifies neatly:

    $$ \delta^L = a - y $$
    
    * In the interest of efficiency and preventing rounding errors, we write a function which directly calculates this value, `error_for_cross_entropy_softmax`, instead of separately calculating $\nabla_aC$ and $\sigma'(z^L)$.

In [12]:
def cost(fn, a, y):
    function = COST_FUNCTIONS.get(fn.lower())
    if function is None:
        raise ValueError(f"unknown function: {fn}")
    return function(a, y)
    
def cost_derivative(cost_fn, a, y):
    function = COST_DERIVATIVES.get(cost_fn)
    if function is None:
        raise ValueError(f"unknown derivative : {function}")
    return function(a, y)
    
def mse(a, y):
    return (0.5 * np.linalg.norm(a - y)**2)

def cross_entropy(a, y):
    epsilon = 1e-10  # added to prevent log(0)
    return -np.sum(y * np.log(a + epsilon))

def mse_derivative(a, y):
    return (a - y)
    
COST_FUNCTIONS = {
    'mse': mse,
    'cross_entropy': cross_entropy
}

COST_DERIVATIVES = {
    mse: mse_derivative
}

In [13]:
# Assuming a softmax activation output layer
def error_for_cross_entropy_softmax(a, y):
    return (a - y)

# <a name="layer-classes"></a> Layer Classes
***
* Neural networks consist of layers.


* In order to structure the code for easy extension (see the [open-closed principle](https://en.wikipedia.org/wiki/Open%E2%80%93closed_principle) of object-oriented programming), we have an abstract class `Layer`, which will act as a superclass for specific types of layers. 
    * In short, this makes it easier to add more types of layers in the future, without duplicating code that provides functionalities which are universal across all layer types.
    
    
* `InputLayer`s have no activation function.

* `DenseLayer`s are layers where all the neurons are connected to all the neurons in the previous and next layer.

In [14]:
class Layer(ABC):
    
    def __init__(self, units=1):
        self.units = units
       
    @abstractmethod
    def layer_activation(self, a_in):
        raise NotImplementedError()

In [15]:
class DenseLayer(Layer):
    
    def __init__(self, units=1, fn="relu"):
        super().__init__(units)
        self.fn = activation_function(fn)
        self.biases = None
        self.weights = None
    
    # Returns weighted_input (z) , activation (a)
    def layer_activation(self, a_in):
        z = np.dot(self.weights, a_in) + self.biases
        a = self.fn(z)
        return z, a
    
    def set_biases(self, biases):
        self.biases = biases
    
    def set_weights(self, weights):
        self.weights = weights

In [16]:
class InputLayer(Layer):
    
    def __init__(self, units=1):
        super().__init__(units)
        
    def activation_function(self, fn):
        raise NotImplementedError("Input layer does not have an activation function.")
    
    def layer_activation(self, a_in):
        return a_in.reshape(a_in.shape[0], 1)

# <a name="neural-network-classes"></a> Neural Network Classes
***
* These classes encapsulate the behaviour and properties of a neural network, making it easier to instantiate, train, and use a neural network.

* Just like with the layer classes, we have an abstract class `NeuralNetwork`, which contains the properties and methods shared by all types of neural networks.
    * This design choice makes it easier to add new classes for other types of neural networks such as CNNs and RNNs.

* `MultiLayerPerceptron`: A Multilayer Perceptron (MLP) is a feed forward neural network (FNN) consisting of dense layers. It is usually seen as the "vanilla" neural network.

* We want creating a neural network to be intuitive. The constructor for the neural network takes in an array of layer objects, so one could create a neural network instance like so: 
        MultiLayerPerceptron([InputLayer(units=784), 
                              DenseLayer(units=300, fn="sigmoid"), 
                              DenseLayer(units=10, fn="softmax")])

### Key methods relating to MLPs:

* **`_forward_prop`**: The mechanism by which the neural network makes a prediction. The input goes into the input layer. Then, calculations are performed on the input as it passes through the layers, all the way to the output layer.


* **`_train_sgd`**: Gradient descent is the algorithm at the heart of "learning". When we say that a neural network is "learning" or "being trained", it refers to running gradient descent on some training data.
    * More specifically, this method implements a variant of gradient descent called **Stochastic Gradient Descent (SGD)**, where the each update of the weights and biases are done based off a mini-batch at a time, as opposed to the entire training set, as is done in **Batch Gradient Descent (BGD)**



* **`_back_prop`**: In updating the weights and biases during gradient descent, we need to calculate certain quantities, namely:

    * $\frac{\partial C}{\partial b^l_j}$: The partial derivative of the cost function $C$ with respect to each bias parameter $b$ in the neural network.
    
    * $\frac{\partial C}{\partial w^l_{jk}}$: The partial derivative of the cost function $C$ with respect to each weight parameter $w$ in the neural network.
    
    * Hence, the backpropropagation algorithm, as implemented by `_back_prop`, goes through the neural network starting from the output layer and ending at the first layer calculating these values.


In [17]:
class NeuralNetwork(ABC):
    
    def __init__(self, layers):
        self.layers = layers
        self.num_layers = len(layers)
               
    def train(self, algorithm, epochs, learning_rate, training_set, validation_set, cost_function, learning_rate_decay_rate=0, augment_images=False, augmented_fraction=0.1, mini_batch_size=None, save_best_parameters=False, seed=None):
        algorithm = algorithm.lower()
        cost_function = COST_FUNCTIONS.get(cost_function.lower())
        if seed:
            np.random.seed(seed)
            
        if augment_images:
            training_set = augment_training_images(training_set, augmented_fraction)
            
        if algorithm == "sgd":
            self._train_sgd(epochs, learning_rate, training_set, validation_set, mini_batch_size, cost_function, learning_rate_decay_rate, save_best_parameters, seed)
        elif algorithm == "bgd":
            self._train_bgd(epochs, learning_rate, training_set, validation_set, cost_function, learning_rate_decay_rate, save_best_parameters)
        else:
            raise ValueError(f"unknown algorithm: {algorithm}")
    
    def get_layer(self, index):
        return self.layers[index]
    
    @abstractmethod
    def _forward_prop(self):
        raise NotImplementedError()
    
    @abstractmethod
    def _back_prop(self):
        raise NotImplementedError()
        
    @abstractmethod
    def _train_sgd(self):
        raise NotImplementedError() 
        
    @abstractmethod
    def _train_bgd(self):
        raise NotImplementedError() 

In [18]:
class MultiLayerPerceptron(NeuralNetwork):
    
    def __init__(self, layers, weight_init="standard_normal", bias_init="standard_normal", constant=None):
        super().__init__(layers)
        self.best_params = None
        for i in range(1, self.num_layers):
            layer = self.layers[i]
            prev_layer = self.layers[i-1]
#             layer.biases = bias_init_function(bias_init, constant)(layer.units)
#             layer.weights = weight_init_function(weight_init)(layer.units, prev_layer.units)
            layer.set_biases(bias_init_function(bias_init, constant)(layer.units))
            layer.set_weights(weight_init_function(weight_init)(layer.units, prev_layer.units))
    
    def get_params(self):
        weights = []
        biases = []
        for layer in self.layers[1:]:
            weights.append(layer.weights)
            biases.append(layer.biases)
        return weights, biases
    
    def set_params(self, weights, biases):
        for i in range(1, self.num_layers):
            layer = self.layers[i]
            layer.set_biases = biases[i-1]
            layer.set_weights = weights[i-1]
    
    def set_best_params(self, weights, biases):
        self.best_params = (weights, biases)
    
    def _forward_prop(self, x):
        activations = []
        weighted_inputs = []  
        input_layer = self.get_layer(0)
        a = input_layer.layer_activation(x)
        activations.append(a) 
        for layer in self.layers[1:]:
            z, a = layer.layer_activation(a)
            activations.append(a)
            weighted_inputs.append(z)
        return weighted_inputs, activations
    
    def predict(self, x):
        weighted_inputs, activations = self._forward_prop(x)
        return activations[-1]
    
    def classify(self, x):
        prediction = self.predict(x)
        classification = np.zeros(prediction.shape)
        one_index = np.argmax(prediction, axis=0)
        classification[one_index] = np.array([1.0])
        return classification
    
    def _train_bgd(self, epochs, learning_rate, training_set, validation_set, cost_function, learning_rate_decay_rate, save_best_parameters, seed=None):
        best_weights = None
        best_biases = None
        best_valid_accuracy = -1.0
        for epoch in range(epochs):
            
            learning_rate = (1 / (1 + learning_rate_decay_rate * epoch)) * learning_rate
            
            self.update_mini_batch(training_set, cost_function, learning_rate)
            
            # Evaluate performance for this epoch
            train_cost = sum([cost_function(self.predict(x), y) for x, y in training_set]) / len(training_set)
            valid_cost = sum([cost_function(self.predict(x), y) for x, y in validation_set]) / len(validation_set)
            print(f"epoch {epoch} complete! | train_cost: {np.around(train_cost, 5)} | train_accuracy: {np.around(self.accuracy(training_set), 5)} | valid_cost: {np.around(valid_cost, 5)} | valid_accuracy: {np.around(self.accuracy(validation_set), 5)}\n")
            
            if save_best_parameters:
                if valid_accuracy > best_valid_accuracy:
                    best_weights, best_biases = self.get_params()
                    self.set_best_params(best_weights, best_biases)
                    
    def _train_sgd(self, epochs, learning_rate, training_set, validation_set, mini_batch_size, cost_function, learning_rate_decay_rate, save_best_parameters, seed=None):
        best_weights = None
        best_biases = None
        best_valid_accuracy = -1.0
        for epoch in range(epochs):
            if seed:
                np.random.seed(seed + epoch) 
            mini_batches = self.make_mini_batches(training_set, mini_batch_size)
            
            learning_rate = (1 / (1 + learning_rate_decay_rate * epoch)) * learning_rate
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, cost_function, learning_rate)
            
            # Evaluate performance for this epoch
            train_cost = sum([cost_function(self.predict(x), y) for x, y in training_set]) / len(training_set)
            valid_cost = sum([cost_function(self.predict(x), y) for x, y in validation_set]) / len(validation_set)
            train_accuracy = self.accuracy(training_set)
            valid_accuracy = self.accuracy(validation_set)
            print(f"epoch {epoch} complete! | train_cost: {np.around(train_cost, 5)} | train_accuracy: {np.around(train_accuracy, 5)} | valid_cost: {np.around(valid_cost, 5)} | valid_accuracy: {np.around(valid_accuracy, 5)}\n")
            
            if save_best_parameters:
                if valid_accuracy > best_valid_accuracy:
                    best_weights, best_biases = self.get_params()
                    self.set_best_params(best_weights, best_biases)
    
    def _back_prop(self, x, y, cost_function):
        nablaC_b = [np.zeros(layer.biases.shape) for layer in self.layers[1:]]
        nablaC_w = [np.zeros(layer.weights.shape) for layer in self.layers[1:]]
        delta = [np.zeros(layer.biases.shape) for layer in self.layers[1:]]
        output_activation_fn = self.get_layer(-1).fn
        
        # Forward pass
        weighted_inputs, activations = self._forward_prop(x)
        
        # Compute error for output layer
        if ((cost_function == cross_entropy) and (output_activation_fn == softmax)):
            delta[-1] = error_for_cross_entropy_softmax(activations[-1], y)
        else:
            delta[-1] = cost_derivative(cost_function, activations[-1], y) * \
            activation_derivative(self.get_layer(-1).fn, weighted_inputs[-1])        
        nablaC_b[-1] = delta[-1]
        nablaC_w[-1] = np.dot(delta[-1], activations[-2].transpose())  
        
        # Backpropagate error:
        for i in range(2, self.num_layers):
            delta[-i] = np.dot(self.get_layer(-i+1).weights.transpose(), delta[-i+1]) * \
            activation_derivative(self.get_layer(-i).fn, weighted_inputs[-i])
            nablaC_b[-i] = delta[-i]
            nablaC_w[-i] = np.dot(delta[-i], activations[-i-1].transpose())
            
        return nablaC_b, nablaC_w
        
    # Calculates the average gradients in one mini-batch, then updates the parameters
    # 1. Loop through the mini-batch, summing all the partial derivatives w.r.t all w, b 
    # 2. Divide the partial derivatives by the mini_batch_size and multiply by learning_rate
    # 3. decrement this value from the parameters
    def update_mini_batch(self, mini_batch, cost_function, learning_rate):
        sum_nablaC_b = [np.zeros(layer.biases.shape) for layer in self.layers[1:]]
        sum_nablaC_w = [np.zeros(layer.weights.shape) for layer in self.layers[1:]]
        
        for x,y in mini_batch:
            nablaC_b, nablaC_w = self._back_prop(x, y, cost_function)
            sum_nablaC_b = [snb + nb for snb, nb in zip(sum_nablaC_b, nablaC_b)]
            sum_nablaC_w = [snw + nw for snw, nw in zip(sum_nablaC_w, nablaC_w)]
            
        # Update step:
        # w = w - (learning_rate)*(sum_nabla_w / mini_batch_size)
        # b = b - (learning_rate)*(sum_nabla_b / mini_batch_size)
        for i in range(1, self.num_layers):
            self.get_layer(i).weights = self.get_layer(i).weights - learning_rate * (sum_nablaC_w[i-1] / len(mini_batch))
            self.get_layer(i).biases = self.get_layer(i).biases - learning_rate * (sum_nablaC_b[i-1] / len(mini_batch))
            
    def make_mini_batches(self, training_set, mini_batch_size):
        residual_batch_size = len(training_set) % mini_batch_size
        num_mini_batches = len(training_set) // mini_batch_size
        
        shuffled_training_set = np.copy(training_set)
        np.random.shuffle(shuffled_training_set)
        
        if (residual_batch_size > 0):
            mini_batches = np.split(shuffled_training_set[:-residual_batch_size], num_mini_batches)
            residual_batch = shuffled_training_set[-residual_batch_size:]
            mini_batches.append(residual_batch)
        else:
            mini_batches = np.split(shuffled_training_set, num_mini_batches)
        
        return mini_batches
    
    def accuracy(self, test_set):
        correct_predictions = 0
        for x, y in test_set:
            predicted_class = np.argmax(self.classify(x))
            actual_class = np.argmax(y)
            if actual_class == predicted_class:
                correct_predictions += 1
        return correct_predictions / len(test_set)

# <a name="creating-a-digit-classifier"></a> Creating a Digit Classifier
***
* **Input Layer**: 784 units since our input images are vectors with 784 elements. One unit for one pixel.

* **Hidden Layer**: There are no strict rules for how many hidden units a neural network should have. Rather, various heuristics along with empirical testing are used to determine a suitable number.

* **Output Layer**: Since there are 10 categories in our classification problem (0-9), we will have 10 units in the output layer. The softmax activation function turns the output of the neural network into a probability distribution: a 10-element vector, where each element describes the probability that the input image depicts that digit.

In [19]:
net = MultiLayerPerceptron([InputLayer(units=784), 
                                DenseLayer(units=300, fn="relu"),
                                DenseLayer(units=10, fn="softmax")],
                               weight_init="he_initialization",
                               bias_init="small_constant")

In [20]:
# Accuracy before training
net.accuracy(validation_set)

0.05523809523809524

In [21]:
# Train a model with the following specifications:
# - 15 epochs
# - learning rate of 0.03
# - mini batch size of 32
# - augment 1% of the training set
# - save the parameters that led to the best performance in the neural network object instance

net.train("sgd", 15, 0.03, training_set, 
                          validation_set,
                          "cross_entropy",
                          mini_batch_size=32,
                          augment_images=True,
                          augmented_fraction=0.01,
                          save_best_parameters=True)

training_set.shape: (29400, 2)

augmented_training_set.shape: (29694, 2)

epoch 0 complete! | train_cost: 0.32754 | train_accuracy: 0.90826 | valid_cost: 0.33974 | valid_accuracy: 0.90286

epoch 1 complete! | train_cost: 0.26057 | train_accuracy: 0.92712 | valid_cost: 0.27508 | valid_accuracy: 0.92222

epoch 2 complete! | train_cost: 0.2275 | train_accuracy: 0.93612 | valid_cost: 0.24771 | valid_accuracy: 0.92968

epoch 3 complete! | train_cost: 0.19488 | train_accuracy: 0.94612 | valid_cost: 0.21852 | valid_accuracy: 0.93476

epoch 4 complete! | train_cost: 0.17462 | train_accuracy: 0.95117 | valid_cost: 0.20064 | valid_accuracy: 0.93921

epoch 5 complete! | train_cost: 0.1534 | train_accuracy: 0.95716 | valid_cost: 0.18409 | valid_accuracy: 0.94206

epoch 6 complete! | train_cost: 0.13801 | train_accuracy: 0.96232 | valid_cost: 0.17074 | valid_accuracy: 0.94921

epoch 7 complete! | train_cost: 0.12813 | train_accuracy: 0.96474 | valid_cost: 0.16226 | valid_accuracy: 0.95048

epoch 8 

In [22]:
# Accuracy after training
net.accuracy(validation_set)

0.9625396825396826

In [23]:
# Try classify a random image (rerun this cell to classify a different image!)
my_training_example = validation_set[random.randint(0, 1000)]
my_image = my_training_example[0]
my_label = my_training_example[1]
print(f"prediction: {np.argmax(net.classify(my_image))}\n\n")
visualize_input(my_image)

prediction: 7




,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015686,0.113725,0.458824,0.552941,0.552941,0.552941,0.898039,0.992157,0.949020,0.552941,0.207843,0.113725,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176471,0.615686,0.701961,0.988235,0.988235,0.988235,0.992157,0.988235,0.988235,0.988235,0.992157,0.988235,0.988235,0.988235,0.615686,0.074510,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.074510,0.843137,0.988235,0.992157,0.988235,0.988235,0.988235,0.658824,0.658824,0.658824,0.658824,0.658824,0.733333,0.988235,0.988235,0.992157,0.745098,0.074510,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027451,0.796078,0.988235,0.992157,0.988235,0.988235,0.792157,0.000000,0.000000,0.000000,0.000000,0.000000,0.027451,0.254902,0.941176,0.992157,0.988235,0.219608,0.000000,0.000000,0.000000,0.000000


## <a name="Comparing-back-prop-to-tensorflow"></a> Comparing `_back_prop` to TensorFlow's backpropagation implementation
***

In [24]:
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [25]:
x = training_set[0][0]
y = training_set[0][1]

In [26]:
nablaC_b, nablaC_w = net._back_prop(x, y, cross_entropy)

my_gradients = [nablaC_w[0].T,
                nablaC_b[0].reshape(-1),
                nablaC_w[1].T,
                nablaC_b[1].reshape(-1)]

In [27]:
# Convert x to a batch of one sample:
x = tf.expand_dims(x, 0)
y = y.reshape(1, 10)

In [28]:
# Define the neural network in TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(300, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [29]:
# Initialize model weights to match your custom network (if needed)

your_weights = [net.get_layer(1).weights.T, 
                net.get_layer(1).biases.reshape(-1),
                net.get_layer(2).weights.T,
                net.get_layer(2).biases.reshape(-1)]


model.set_weights(your_weights)

In [30]:
# Compute gradients using TensorFlow
with tf.GradientTape() as tape:
    predictions = model(x)  # x should be the input tensor
    loss = tf.keras.losses.categorical_crossentropy(y, predictions)  # assuming y is one-hot encoded

grads = tape.gradient(loss, model.trainable_variables)

In [31]:
# Compare gradients
for tf_grad, my_grad in zip(grads, my_gradients):
    difference = tf.norm(tf_grad - my_grad).numpy()
    print(f"Gradient difference: {difference}")

Gradient difference: 2.701109913516575e-08
Gradient difference: 2.6649289441849078e-09
Gradient difference: 1.4929842251376613e-08
Gradient difference: 1.1660429288795626e-09
